by [__Tiago Dias__](https://www.linkedin.com/in/diasctiago/) 

# NLP com scikit-learn

## Importando bibliotecas

In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from joblib import dump, load
import matplotlib.pyplot as plt
#from wordcloud import WordCloud


[nltk_data] Downloading package stopwords to /home/dias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Carregando dataset

In [10]:
df = pd.read_csv('catalogo.csv',delimiter=';')
df.head(2)

,Texto,Classificação,Título,Link
0,dia nacional do combate às drogas e ao alcoolismo,Consumo de Álcool,Um panorama através da pandemia do Covid-19,https://healthlake.com.br/artigos/consumo-de-a...
1,abuso de álcool e drogas como problema de saúd...,Consumo de Álcool,Um panorama através da pandemia do Covid-19,https://healthlake.com.br/artigos/consumo-de-a...


## Possíveis classificações

In [11]:
df_classificador = df[['Classificação','Título','Link']].drop_duplicates().reset_index(drop=True).copy()
df_classificador.head(10)

,Classificação,Título,Link
0,Consumo de Álcool,Um panorama através da pandemia do Covid-19,https://healthlake.com.br/artigos/consumo-de-a...
1,Consumo de Álcool,Análise Fevereiro – Alcoolismo,https://public.tableau.com/profile/health.lake...
2,Covid,Casos de COVID-19 no Brasil,https://public.tableau.com/profile/rcsalgueiro...
3,Covid,Evolução de mortes causadas pelo COVID-19 em 2020,https://public.tableau.com/profile/health.lake...
4,Doação de Sangue,Doação de sangue: uma análise da nossa corrent...,https://healthlake.com.br/artigos/doacao-de-sa...
5,Hepatite C,Combate à Hepatite C no Brasil: vamos retornar...,https://healthlake.com.br/artigos/-HqCombate%2...
6,Perfil de Consumo Brasileiro,Uma visão analítica do perfil de consumo no Br...,https://healthlake.com.br/artigos/uma-visao-an...
7,SUS,Razões da eficiência do SUS no controle da pan...,https://healthlake.com.br/artigos/razoes-da-ef...
8,Tratamentos Oncológicos,Os Danos Silenciosos da COVID-19,https://healthlake.com.br/artigos/os-danos-sil...
9,Violência Contra Mulher,Violência Doméstica em Tempos de Pandemia Viol...,https://healthlake.com.br/artigos/violencia-do...


In [12]:
df['Classificação'].value_counts()

Consumo de Álcool               20
Covid                           20
Doação de Sangue                20
Hepatite C                      20
Perfil de Consumo Brasileiro    20
SUS                             20
Tratamentos Oncológicos         20
Violência Contra Mulher         20
Name: Classificação, dtype: int64

## Criando uma nova coluna

In [13]:
df['nv_Texto'] = df['Texto'].copy()

## Pontuação

In [14]:
df['nv_Texto'] = df['nv_Texto'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
df[['Texto','nv_Texto']][df['Texto'].str.contains(',')]

,Texto,nv_Texto
13,Quantidade de ocorrências por estado relaciona...,Quantidade de ocorrências por estado relaciona...
20,Qual é a evolução acumulada de casos de covid ...,Qual é a evolução acumulada de casos de covid ...
37,"Evolução diária de covid, média móvel",Evolução diária de covid média móvel
38,"Acúmulo de casos de covid por país, por dia",Acúmulo de casos de covid por país por dia
80,"dados do perfil de consumo no Brasil pré, dura...",dados do perfil de consumo no Brasil pré dura...
81,"informações sobre os brasileiros pré, durante ...",informações sobre os brasileiros pré durante ...
127,"números de tratamentos, tanto de Quimioterapia...",números de tratamentos tanto de Quimioterapia...
133,"Dados sobre oncologia antes, durante e depois ...",Dados sobre oncologia antes durante e depois ...
140,"Lei Maria da Penha, n. 11.340, de 7 de agosto ...",Lei Maria da Penha n 11 340 de 7 de agosto ...
141,cinco tipos de violência doméstica e familiar ...,cinco tipos de violência doméstica e familiar ...


## Caracteres Especiais

In [18]:
df['nv_Texto'] = df['nv_Texto'].str.replace('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
df[['Texto','nv_Texto']][df['Texto'].str.contains('-')]

,Texto,nv_Texto
29,Informações sobre COVID-19,Informações sobre COVID
30,Dados sobre COVID-19 no Brasil,Dados sobre COVID no Brasil
84,Os impactos da crise financeira internacional ...,Os impactos da crise financeira internacional ...
90,dados socioeconômicos segundo o plano de fundo...,dados socioeconômicos segundo o plano de fundo...
100,eficiência do SUS no controle da pandemia de C...,eficiência do SUS no controle da pandemia de C...


## Números

In [19]:
df['nv_Texto'] = df['nv_Texto'].str.replace('[0-9]+', '', regex=True)
df[['Texto','nv_Texto']][df['Texto'].str.contains('7')]

,Texto,nv_Texto
140,"Lei Maria da Penha, n. 11.340, de 7 de agosto ...",Lei Maria da Penha n de de agosto de


## Stop Words

In [20]:
str(stopwords.words('portuguese'))

"['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'esti

In [21]:
stop_words = stopwords.words('portuguese')

## Tokenização

In [22]:
# Não faz parte do problema, somente para visualização
frase = 'Texto exemplo tokenização'
nltk.word_tokenize(frase)

['Texto', 'exemplo', 'tokenização']

## CountVectorizer

In [23]:
# Criação da função CountVectorizer
cvt = CountVectorizer(strip_accents='ascii', lowercase=True, stop_words=stop_words)

In [24]:
# Não faz parte do problema, somente para visualização
# Seleção de dois itens da nossa base de dados
exemplo_descricao = df['nv_Texto'][:2]
exemplo_descricao.values

array(['dia nacional do combate às drogas e ao alcoolismo',
       'abuso de álcool e drogas como problema de saúde pública'],
      dtype=object)

In [25]:
# Não faz parte do problema, somente para visualização
# Transformação dos dois itens em vetores binários
exemplo_descricao_cvt = cvt.fit_transform(exemplo_descricao)
exemplo_descricao_cvt.toarray()

/home/dias/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'ha', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'ja', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'so', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


array([[0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 0, 1, 1, 1]])

In [26]:
X_cvt = cvt.fit_transform(df['nv_Texto'])

In [27]:
# Criação da função TfidfTransformer
tfi = TfidfTransformer(use_idf=True)

In [28]:
# Não faz parte do problema, somente para visualização
# Transformação dos exemplos com a normalização tf-idf
exemplo_descricao_tfi = tfi.fit_transform(exemplo_descricao_cvt)
exemplo_descricao_tfi.toarray()

array([[0.        , 0.        , 0.47107781, 0.47107781, 0.47107781,
        0.33517574, 0.47107781, 0.        , 0.        , 0.        ],
       [0.4261596 , 0.4261596 , 0.        , 0.        , 0.        ,
        0.30321606, 0.        , 0.4261596 , 0.4261596 , 0.4261596 ]])

In [29]:
X_tfi = tfi.fit_transform(X_cvt)

In [30]:
# A entrada será a transformação de vetores com a normalização tf-idf
entrada = X_tfi
# A saida será os departamentos
saida = df['Classificação']
# Separando 20% dos dados para teste
X_train, X_test, y_train, y_test = train_test_split(entrada, saida, test_size=0.3)

## Modelo LinearSVC

In [31]:
# Criando modelo
clf = LinearSVC()
# Treinamento do modelo
clf.fit(X_train, y_train)

LinearSVC()

## Predição e avaliação do modelo

In [32]:
# Realizando a predição
resultado = clf.predict(X_test)
# Avaliando o modelo
print('Acurácia: {:.2f}'.format(metrics.accuracy_score(y_test, resultado)))

Acurácia: 0.92


In [34]:
# Avaliação completa
print(metrics.classification_report(y_test, resultado))

                              precision    recall  f1-score   support

           Consumo de Álcool       0.86      1.00      0.92         6
                       Covid       0.86      1.00      0.92         6
            Doação de Sangue       1.00      1.00      1.00         7
                  Hepatite C       0.80      1.00      0.89         4
Perfil de Consumo Brasileiro       1.00      0.56      0.71         9
                         SUS       1.00      1.00      1.00         7
     Tratamentos Oncológicos       1.00      1.00      1.00         3
     Violência Contra Mulher       0.86      1.00      0.92         6

                    accuracy                           0.92        48
                   macro avg       0.92      0.94      0.92        48
                weighted avg       0.93      0.92      0.91        48



## Utilização do modelo

A função para utilização do modelo, recebe uma descrição e retorna o departamento para essa descrição.


In [35]:
def novo_texto(nv_Texto):
  novo_cvt = cvt.transform(pd.Series(nv_Texto))
  novo_tfi = tfi.transform(novo_cvt)
  classificacao = clf.predict(novo_tfi)[0]
  return classificacao

No final o nosso resultado mostrando (Produto: descrição informada Departamento: departamento previsto). Portanto, podemos ver a assertividade do nosso modelo na prática.

In [36]:
# Lista de exemplos de novos produtos
Textos = ['Dados sobre Cancer','Dados sobre consumo de bebidas']
# Loop for para fazer a predição do departamento de novos produtos
for Texto in Textos:
  print('Texto:', Texto, 'Classificação:', novo_texto(Texto))

Texto: Dados sobre Cancer Classificação: Tratamentos Oncológicos
Texto: Dados sobre consumo de bebidas Classificação: Consumo de Álcool


## Criando Pipeline do modelo

In [37]:
entrada = df['nv_Texto']
# A saida será os departamentos
saida = df['Classificação']

X_train, X_test, y_train, y_test = train_test_split(entrada, saida, test_size=0.3)

pipe = Pipeline([
                ('cvt', CountVectorizer(strip_accents='ascii', lowercase=True, stop_words=stop_words)), 
                ('tfi', TfidfTransformer(use_idf=True)), 
                ('clf', LinearSVC())
                ])

In [38]:
pipe.fit(X_train, y_train)

/home/dias/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'ha', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'ja', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'so', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('cvt',
                 CountVectorizer(stop_words=['de', 'a', 'o', 'que', 'e', 'é',
                                             'do', 'da', 'em', 'um', 'para',
                                             'com', 'não', 'uma', 'os', 'no',
                                             'se', 'na', 'por', 'mais', 'as',
                                             'dos', 'como', 'mas', 'ao', 'ele',
                                             'das', 'à', 'seu', 'sua', ...],
                                 strip_accents='ascii')),
                ('tfi', TfidfTransformer()), ('clf', LinearSVC())])

In [39]:
pipe.score(X_test, y_test)

0.9166666666666666

In [40]:
# Lista de exemplos de novos produtos
Textos = ['Panorâma da violência contra a mulher','Dados sobre Cancer','Dados sobre consumo de bebidas']
# Loop for para fazer a predição do departamento de novos produtos
for Texto in Textos:
  print('Texto:', Texto, 'Classificação:', pipe.predict([Texto])[0])

Texto: Panorâma da violência contra a mulher Classificação: Violência Contra Mulher
Texto: Dados sobre Cancer Classificação: Tratamentos Oncológicos
Texto: Dados sobre consumo de bebidas Classificação: Consumo de Álcool


## Exportando Pipeline do modelo

In [41]:
dump(pipe, 'classificador.joblib') 

['classificador.joblib']

## Load Pipeline do modelo

In [42]:
classificador = load('classificador.joblib') 

In [43]:
# Lista de exemplos de novos produtos
Textos = ['Panorâma da violência contra a mulher',
          'Dados sobre doenças oncológicas',
          'Dados sobre consumo de bebidas',
          'Covid no Brasil']
# Loop for para fazer a predição do departamento de novos produtos
for Texto in Textos:
  print('Texto:', Texto, 'Classificação:', classificador.predict([Texto])[0])

Texto: Panorâma da violência contra a mulher Classificação: Violência Contra Mulher
Texto: Dados sobre doenças oncológicas Classificação: Tratamentos Oncológicos
Texto: Dados sobre consumo de bebidas Classificação: Consumo de Álcool
Texto: Covid no Brasil Classificação: Covid


/home/dias/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'ha', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'ja', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'so', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


## Action Rasa

In [44]:
Texto = [{'Texto': 'Panorâma da violência# contra a mulher 2020!'}]

In [45]:
df_texto = pd.DataFrame(Texto)
df_texto

,Texto
0,Panorâma da violência# contra a mulher 2020!


In [46]:
def limpa_texto(df_texto):
    df_texto['nv_Texto'] = df_texto['Texto'].copy()
    df_texto['nv_Texto'] = df_texto['nv_Texto'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
    df_texto['nv_Texto'] = df_texto['nv_Texto'].str.replace('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
    df_texto['nv_Texto'] = df_texto['nv_Texto'].str.replace('[0-9]+', '', regex=True)
    return df_texto

In [47]:
df_texto = limpa_texto(df_texto)
df_texto

,Texto,nv_Texto
0,Panorâma da violência# contra a mulher 2020!,Panorâma da violência contra a mulher


In [48]:
df_texto['nv_Texto']

0    Panorâma da violência  contra a mulher  
Name: nv_Texto, dtype: object

In [49]:
classifica = classificador.predict(df_texto['nv_Texto'])[0]
classifica

'Violência Contra Mulher'

In [50]:
titulo = df_classificador.query('Classificação == @classifica')['Título'].values[0]
titulo

'Violência Doméstica em Tempos de Pandemia Violência Contra Mulheres'

In [51]:
link = df_classificador.query('Classificação == @classifica')['Link'].values[0]
link

'https://healthlake.com.br/artigos/violencia-domestica-em-tempos-de-pandemia-violencia-contra-mulheres'

In [52]:
str_result = 'Título: ' + titulo + ', no link: ' + link
str_result

'Título: Violência Doméstica em Tempos de Pandemia Violência Contra Mulheres, no link: https://healthlake.com.br/artigos/violencia-domestica-em-tempos-de-pandemia-violencia-contra-mulheres'